# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 1 2022 11:09AM,238531,10,8091178,Bio-PRF,Released
1,Jun 1 2022 11:08AM,238529,12,HH-34649,Hush Hush,Released
2,Jun 1 2022 11:08AM,238529,12,HH-34650,Hush Hush,Released
3,Jun 1 2022 11:08AM,238529,12,HH-34651,Hush Hush,Released
4,Jun 1 2022 11:08AM,238529,12,HH-34652,Hush Hush,Released
5,Jun 1 2022 11:08AM,238529,12,HH-34653,Hush Hush,Released
6,Jun 1 2022 11:08AM,238529,12,HH-34654,Hush Hush,Released
7,Jun 1 2022 11:08AM,238529,12,HH-34655,Hush Hush,Released
8,Jun 1 2022 11:08AM,238529,12,HH-34656,Hush Hush,Released
9,Jun 1 2022 11:08AM,238529,12,HH-34657,Hush Hush,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
58,238527,Released,36
59,238528,Released,16
60,238529,Released,60
61,238530,Released,1
62,238531,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238527,NaN,NaN,36.0
238528,NaN,NaN,16.0
238529,NaN,NaN,60.0
238530,NaN,NaN,1.0
238531,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238318,0.0,1.0,19.0
238342,0.0,0.0,1.0
238343,0.0,0.0,1.0
238426,0.0,1.0,0.0
238428,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
238318,0,1,19
238342,0,0,1
238343,0,0,1
238426,0,1,0
238428,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238318,0,1,19
1,238342,0,0,1
2,238343,0,0,1
3,238426,0,1,0
4,238428,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,238318,,1,19
1,238342,,,1
2,238343,,,1
3,238426,,1,
4,238428,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 1 2022 11:09AM,238531,10,Bio-PRF
1,Jun 1 2022 11:08AM,238529,12,Hush Hush
61,Jun 1 2022 11:06AM,238528,10,Bio-PRF
77,Jun 1 2022 10:54AM,238530,10,ISDIN Corporation
78,Jun 1 2022 10:51AM,238527,10,ISDIN Corporation
114,Jun 1 2022 10:46AM,238526,16,"SD Head USA, LLC"
115,Jun 1 2022 10:43AM,238525,19,"Emersa Waterbox, LLC"
116,Jun 1 2022 10:39AM,238524,16,"SD Head USA, LLC"
117,Jun 1 2022 10:32AM,238523,10,Eywa Pharma Inc.
122,Jun 1 2022 10:30AM,238522,18,"ClearSpec, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 1 2022 11:09AM,238531,10,Bio-PRF,,,1
1,Jun 1 2022 11:08AM,238529,12,Hush Hush,,,60
2,Jun 1 2022 11:06AM,238528,10,Bio-PRF,,,16
3,Jun 1 2022 10:54AM,238530,10,ISDIN Corporation,,,1
4,Jun 1 2022 10:51AM,238527,10,ISDIN Corporation,,,36
5,Jun 1 2022 10:46AM,238526,16,"SD Head USA, LLC",,,1
6,Jun 1 2022 10:43AM,238525,19,"Emersa Waterbox, LLC",,,1
7,Jun 1 2022 10:39AM,238524,16,"SD Head USA, LLC",,,1
8,Jun 1 2022 10:32AM,238523,10,Eywa Pharma Inc.,,,5
9,Jun 1 2022 10:30AM,238522,18,"ClearSpec, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 11:09AM,238531,10,Bio-PRF,1,,
1,Jun 1 2022 11:08AM,238529,12,Hush Hush,60,,
2,Jun 1 2022 11:06AM,238528,10,Bio-PRF,16,,
3,Jun 1 2022 10:54AM,238530,10,ISDIN Corporation,1,,
4,Jun 1 2022 10:51AM,238527,10,ISDIN Corporation,36,,
5,Jun 1 2022 10:46AM,238526,16,"SD Head USA, LLC",1,,
6,Jun 1 2022 10:43AM,238525,19,"Emersa Waterbox, LLC",1,,
7,Jun 1 2022 10:39AM,238524,16,"SD Head USA, LLC",1,,
8,Jun 1 2022 10:32AM,238523,10,Eywa Pharma Inc.,5,,
9,Jun 1 2022 10:30AM,238522,18,"ClearSpec, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 11:09AM,238531,10,Bio-PRF,1,,
1,Jun 1 2022 11:08AM,238529,12,Hush Hush,60,,
2,Jun 1 2022 11:06AM,238528,10,Bio-PRF,16,,
3,Jun 1 2022 10:54AM,238530,10,ISDIN Corporation,1,,
4,Jun 1 2022 10:51AM,238527,10,ISDIN Corporation,36,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 11:09AM,238531,10,Bio-PRF,1.0,NaN,NaN
1,Jun 1 2022 11:08AM,238529,12,Hush Hush,60.0,NaN,NaN
2,Jun 1 2022 11:06AM,238528,10,Bio-PRF,16.0,NaN,NaN
3,Jun 1 2022 10:54AM,238530,10,ISDIN Corporation,1.0,NaN,NaN
4,Jun 1 2022 10:51AM,238527,10,ISDIN Corporation,36.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 1 2022 11:09AM,238531,10,Bio-PRF,1.0,0.0,0.0
1,Jun 1 2022 11:08AM,238529,12,Hush Hush,60.0,0.0,0.0
2,Jun 1 2022 11:06AM,238528,10,Bio-PRF,16.0,0.0,0.0
3,Jun 1 2022 10:54AM,238530,10,ISDIN Corporation,1.0,0.0,0.0
4,Jun 1 2022 10:51AM,238527,10,ISDIN Corporation,36.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4531550,250.0,13.0,15.0
102,715387,3.0,0.0,0.0
12,715498,62.0,1.0,1.0
15,1669050,75.0,29.0,0.0
16,715536,2.0,1.0,0.0
18,953633,3.0,1.0,0.0
19,477020,14.0,14.0,1.0
20,1192430,35.0,37.0,16.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4531550,250.0,13.0,15.0
1,102,715387,3.0,0.0,0.0
2,12,715498,62.0,1.0,1.0
3,15,1669050,75.0,29.0,0.0
4,16,715536,2.0,1.0,0.0
5,18,953633,3.0,1.0,0.0
6,19,477020,14.0,14.0,1.0
7,20,1192430,35.0,37.0,16.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,250.0,13.0,15.0
1,102,3.0,0.0,0.0
2,12,62.0,1.0,1.0
3,15,75.0,29.0,0.0
4,16,2.0,1.0,0.0
5,18,3.0,1.0,0.0
6,19,14.0,14.0,1.0
7,20,35.0,37.0,16.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,250.0
1,102,Released,3.0
2,12,Released,62.0
3,15,Released,75.0
4,16,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,18,19,20
Status,,,,,,,,
Completed,15.0,0.0,1.0,0.0,0.0,0.0,1.0,16.0
Executing,13.0,0.0,1.0,29.0,1.0,1.0,14.0,37.0
Released,250.0,3.0,62.0,75.0,2.0,3.0,14.0,35.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,18,19,20
0,Completed,15.0,0.0,1.0,0.0,0.0,0.0,1.0,16.0
1,Executing,13.0,0.0,1.0,29.0,1.0,1.0,14.0,37.0
2,Released,250.0,3.0,62.0,75.0,2.0,3.0,14.0,35.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,18,19,20
0,Completed,15.0,0.0,1.0,0.0,0.0,0.0,1.0,16.0
1,Executing,13.0,0.0,1.0,29.0,1.0,1.0,14.0,37.0
2,Released,250.0,3.0,62.0,75.0,2.0,3.0,14.0,35.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()